### Import libraries

In [1]:
import pandas as pd

import camelot

import pdfplumber

import os

### Define Function to scroll through each file in the folder

In [2]:
def process_all_pdfs_in_folder(source_path, sink_path, desired_disease):
    year_df = []
    for filename in os.listdir(source_path):
        if filename.lower().endswith('.pdf'):
            pdf_path = os.path.join(source_path, filename)
            print(f"Processando o arquivo: {pdf_path}")
            week_df = extract_and_process_tables(pdf_path, desired_disease, filename)
            year_df.append(week_df)
            
    if year_df:
        concatenated_df = pd.concat(year_df, ignore_index=True)

        pdf_filename = source_path.rsplit('/', 1)[-1]
        print("Salvando")
        csv_filename = os.path.splitext(pdf_filename)[0] + '.csv'
        output_path = os.path.join(sink_path, csv_filename)
        concatenated_df.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"Arquivo salvo: {csv_filename}")
            

### Define function to scroll through each page/table in selected file

In [3]:
def extract_and_process_tables(pdf_path, desired_disease, filename):
    all_dfs = []
    pdf = pdfplumber.open(pdf_path)

    for page_num in range(len(pdf.pages)):
        try:
            tables = camelot.read_pdf(pdf_path, pages=str(page_num + 1), flavor='stream')

            if tables:
                df = tables[0].df
                page = pdf.pages[page_num]
                table = page.extract_table()
                df_header = pd.DataFrame(table)
                                
                if (not df_header.empty 
                    and df_header.shape[0] > 1
                    and df_header.shape[1] > 1
                    and df_header.iloc[0, 1] is not None 
                    and isinstance(df_header.iloc[0, 1], str)
                    and desired_disease in df_header.iloc[0, 1]):   
                    
                                        
                    if filename[3:5] == '01':
                        
                        df_all = process_table_11(df, df_header, filename)

                    else:
                        
                        df_all = process_table_13(df, df_header, filename)
                            
                        
                    if not df_all.empty:
                        all_dfs.append(df_all)
#                         print(all_dfs)
                        print(f"Processando a página {page_num + 1} com a tabela desejada.")

        except Exception as e:
            print(f"Erro ao processar a página {page_num + 1}: {e}")

    pdf.close()

    if all_dfs:
        final_df = pd.concat(all_dfs, ignore_index=True)
    else:
        final_df = pd.DataFrame()


    return final_df

### Define function to transform selected table in readable data, with 13 columns

In [4]:
def process_table_13(df, df_header, filename):
    if df.empty or df_header.empty:
        print("DataFrame ou cabeçalho vazio.")
        return pd.DataFrame() 
    
    weeknumber = filename[3:5]

    df = df.iloc[4:].reset_index(drop=True)
    df.columns = df.iloc[0]
    df = df[1:]

    col_names = [f'Coluna{i+1}' for i in range(len(df.columns))]
    df.columns = col_names

    for col in col_names[1:13]:
        df[col] = df[col].str.replace(' ', '')

    first_col_name = df.columns[0]
    df = df[~df[first_col_name].str.match(r'^(TOTAL|FUENTE.*)')]

    indices_para_converter = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
    for idx in indices_para_converter:
        if idx < len(df.columns):  # Verifica se o índice está dentro dos limites
            df.iloc[:, idx] = pd.to_numeric(df.iloc[:, idx], errors='coerce').fillna(0).astype(int)

    df1 = df.iloc[:, 0:5].copy()
    if len(df_header) > 1 and len(df_header.columns) > 1:
        df1['disease'] = df_header.iloc[0, 1].replace('\n', ' ')
        df1['year'] = df_header.iloc[1, 1].replace('\n', ' ')
    else:
        df1['disease'] = ''
        df1['year'] = ''

    df1['MF'] = df1.get('Coluna3', 0) + df1.get('Coluna4', 0)
    df1['Acum_Year'] = df_header.iloc[1, 4].replace('\n', ' ') if len(df_header) > 1 and len(df_header.columns) > 4 else ''
    
    df1['week'] = weeknumber

    novos_nomes = [
        df_header.iloc[0, 0].replace('\n', ' ') if len(df_header) > 0 and len(df_header.columns) > 0 else '',
        df_header.iloc[2, 1].replace('\n', ' ') if len(df_header) > 2 and len(df_header.columns) > 1 else '',
        df_header.iloc[3, 2].replace('\n', ' ') if len(df_header) > 3 and len(df_header.columns) > 2 else '',
        df_header.iloc[3, 3].replace('\n', ' ') if len(df_header) > 3 and len(df_header.columns) > 3 else '',
        df_header.iloc[2, 4].replace('\n', ' ') if len(df_header) > 2 and len(df_header.columns) > 4 else ''
    ]
    
    df1.columns = novos_nomes + list(df1.columns[5:])
    column_order = [0, 1, 2, 3, 7, 4, 8, 5, 6, 9]
    df1 = df1.iloc[:, column_order]

    df2 = df.iloc[:, [0] + list(range(5, 9))].copy()
    if len(df_header) > 0 and len(df_header.columns) > 5:
        df2['disease'] = df_header.iloc[0, 5].replace('\n', ' ')
        df2['year'] = df_header.iloc[1, 4].replace('\n', ' ')
    else:
        df2['disease'] = ''
        df2['year'] = ''

    df2['MF'] = df2.get('Coluna7', 0) + df2.get('Coluna8', 0)
    df2['Acum_Year'] = df_header.iloc[1, 8].replace('\n', ' ') if len(df_header) > 1 and len(df_header.columns) > 8 else ''
    
    df2['week'] = weeknumber

    df2.columns = novos_nomes + list(df2.columns[5:])
    df2 = df2.iloc[:, column_order]

    df3 = df.iloc[:, [0] + list(range(9, 13))].copy()
    if len(df_header) > 0 and len(df_header.columns) > 9:
        df3['disease'] = df_header.iloc[0, 9].replace('\n', ' ')
        df3['year'] = df_header.iloc[1, 9].replace('\n', ' ')
    else:
        df3['disease'] = ''
        df3['year'] = ''

    df3['MF'] = df3.get('Coluna11', 0) + df3.get('Coluna12', 0)
    df3['Acum_Year'] = df_header.iloc[1, 12].replace('\n', ' ') if len(df_header) > 1 and len(df_header.columns) > 12 else ''
    
    df3['week'] = weeknumber

    df3.columns = novos_nomes + list(df3.columns[5:])
    df3 = df3.iloc[:, column_order]

    df_all = pd.concat([df1, df2, df3], axis=0, ignore_index=True)
    

    return df_all

### Define function to transform selected table in readable data, with 11 columns

In [5]:
def process_table_11(df, df_header, filename):
    if df.empty or df_header.empty:
        print("DataFrame ou cabeçalho vazio.")
        return pd.DataFrame() 
    
    weeknumber = filename[3:5]

    df = df.iloc[4:].reset_index(drop=True)
    df.columns = df.iloc[0]
    df = df[1:]

    col_names = [f'Coluna{i+1}' for i in range(len(df.columns))]
    df.columns = col_names

    for col in col_names[1:9]:
        df[col] = df[col].str.replace(' ', '')

    first_col_name = df.columns[0]
    df = df[~df[first_col_name].str.match(r'^(TOTAL|FUENTE.*)')]

    indices_para_converter = [1, 2, 3, 4, 5, 6, 7, 8, 9]
    for idx in indices_para_converter:
        if idx < len(df.columns): 
            df.iloc[:, idx] = pd.to_numeric(df.iloc[:, idx], errors='coerce').fillna(0).astype(int)

    df1 = df.iloc[:, 0:5].copy()
    if len(df_header) > 1 and len(df_header.columns) > 1:
        df1['disease'] = df_header.iloc[0, 1].replace('\n', ' ')
        df1['year'] = df_header.iloc[1, 1].replace('\n', ' ')
    else:
        df1['disease'] = ''
        df1['year'] = ''

    df1['MF'] = df1.get('Coluna3', 0) + df1.get('Coluna4', 0)
    df1['Acum_Year'] = int(df_header.iloc[1, 1].replace('\n', ' ')) - 1

    df1['week'] = weeknumber

    novos_nomes = [
        df_header.iloc[0, 0].replace('\n', ' ') if len(df_header) > 0 and len(df_header.columns) > 0 else '',
        df_header.iloc[2, 1].replace('\n', ' ') if len(df_header) > 2 and len(df_header.columns) > 1 else '',
        df_header.iloc[3, 2].replace('\n', ' ') if len(df_header) > 3 and len(df_header.columns) > 2 else '',
        df_header.iloc[3, 3].replace('\n', ' ') if len(df_header) > 3 and len(df_header.columns) > 3 else '',
        'Acum.'
    ]

    df1.columns = novos_nomes + list(df1.columns[5:])
    column_order = [0, 1, 2, 3, 7, 4, 8, 5, 6, 9]
    df1 = df1.iloc[:, column_order]

    df2 = df.iloc[:, [0] + list(range(5, 9))].copy()
    if len(df_header) > 0 and len(df_header.columns) > 5:
        df2['disease'] = df_header.iloc[0, 4].replace('\n', ' ')
        df2['year'] = df_header.iloc[1, 4].replace('\n', ' ')
    else:
        df2['disease'] = ''
        df2['year'] = ''

    df2['MF'] = df2.get('Coluna7', 0) + df2.get('Coluna8', 0)
    df2['Acum_Year'] = int(df_header.iloc[1, 4].replace('\n', ' ')) - 1

    df2['week'] = weeknumber

    df2.columns = novos_nomes + list(df2.columns[5:])
    df2 = df2.iloc[:, column_order]

    df3 = df.iloc[:, [0] + list(range(8, 14))].copy()
    if len(df_header) > 0 and len(df_header.columns) > 9:
        df3['disease'] = df_header.iloc[0, 7].replace('\n', ' ')
        df3['year'] = df_header.iloc[1, 7].replace('\n', ' ')
    else:
        df3['disease'] = ''
        df3['year'] = ''

    df3['Coluna12'] = pd.to_numeric(df3['Coluna12'], errors='coerce').fillna(0).astype(int)
    df3['Coluna14'] = pd.to_numeric(df3['Coluna14'], errors='coerce').fillna(0).astype(int)

    df3['MF'] = df3.get('Coluna12', 0) + df3.get('Coluna14', 0)
    df3['Acum_Year'] = int(df_header.iloc[1, 7].replace('\n', ' ')) - 1

    df3['week'] = weeknumber

    df3 = df3.drop(['Coluna11','Coluna13'],axis = 1)

    column_order = [0, 2, 3, 4, 1, 5, 6, 7, 8, 9]

    df3 = df3.iloc[:, column_order]

    df3.columns = novos_nomes + list(df3.columns[5:])

    column_order = [0, 1, 2, 3, 7, 4, 8, 5, 6, 9]
    df3 = df3.iloc[:, column_order]

    df_all = pd.concat([df1, df2, df3], axis=0, ignore_index=True)

    return df_all

### Process selected year/disease and save into selected folder

In [7]:

source_path = '/home/pirata/Documents/projects/epidemic_database/files/2015'
sink_path = '/home/pirata/Documents/projects/epidemic_database/bases/Síndrome Febril'
desired_disease = 'Síndrome Febril'
process_all_pdfs_in_folder(source_path, sink_path, desired_disease)